In [45]:
import pandas as pd 
import matplotlib.pyplot as plt 
import seaborn as sns
from datetime import date
from sklearn.ensemble import RandomForestRegressor
from sklearn.metrics import mean_squared_error, r2_score
import numpy as np


sns.set()

In [6]:
faturamento_df = pd.read_csv("data/faturamento_lojas_atuais.csv")
faturamento_df["datetime"] = pd.to_datetime(faturamento_df.ano*100 + faturamento_df.mes, format='%Y%m')
faturamento_df.drop(["mes", "ano"], axis=1, inplace=True)
faturamento_df.head()

,cod_loja,categoria,qtde,receita,datetime
0,1,CATEG_01,301.0,9164.86,2016-01-01
1,1,CATEG_01,187.0,8175.55,2016-02-01
2,1,CATEG_01,242.0,10521.67,2016-03-01
3,1,CATEG_01,104.0,4560.91,2016-04-01
4,1,CATEG_01,100.0,4263.30,2016-05-01


In [7]:
df = faturamento_df.groupby(["cod_loja", "datetime"], as_index=False).sum()
df.head()

,cod_loja,datetime,qtde,receita
0,1,2016-01-01,13034.0,441833.37
1,1,2016-02-01,11411.0,395340.22
2,1,2016-03-01,15030.0,587810.18
3,1,2016-04-01,12726.0,534618.84
4,1,2016-05-01,13978.0,595477.92


In [52]:
cod_loja_gb = df.groupby("cod_loja")
time_window = 3

X_train = []
y_train = []

X_test = []
y_test = []
for cod_loja, loja_df in cod_loja_gb:
    # pula as lojas que não tem os 24 meses
    if loja_df.shape[0] != 24: continue
        
    t = loja_df.sort_values("datetime", ascending=True)
    for i in range(time_window, 24):
        t_df = t.iloc[i-time_window:i+1, :]
        receita = t_df["receita"].values
        if t_df.iloc[time_window, :]["datetime"].month in (1, 2, 3, 4, 5, 6, 7): continue
        if t_df.iloc[time_window, :]["datetime"] in [date(2017,10,1), date(2017,11,1), date(2017,12,1)]:
            X_test.append(receita[0:time_window])
            y_test.append(receita[time_window])
        else:
            X_train.append(receita[0:time_window])
            y_train.append(receita[time_window])

In [48]:
len(X_train), len(y_train), len(X_test), len(y_test) 

(53334, 53334, 8889, 8889)

## primeiro teste

In [53]:
rf = RandomForestRegressor()
rf.fit(X_train, y_train)

RandomForestRegressor(bootstrap=True, ccp_alpha=0.0, criterion='mse',
                      max_depth=None, max_features='auto', max_leaf_nodes=None,
                      max_samples=None, min_impurity_decrease=0.0,
                      min_impurity_split=None, min_samples_leaf=1,
                      min_samples_split=2, min_weight_fraction_leaf=0.0,
                      n_estimators=100, n_jobs=None, oob_score=False,
                      random_state=None, verbose=0, warm_start=False)

In [54]:
y_pred = rf.predict(X_test)
np.sqrt(mean_squared_error(y_test, y_pred))

74389.42703952325